---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    
    # Your Code Here
    #Analyzed this by seeing if the degrees follow a power-law distribution. If not, check with the averange clustering
    #and averange shorted distance if it is SW_L or SW_P, with a high averange shortest distance and a high local clustering 
    #saying that it is SW_L.
    return list(['PA', 'SW_L', 'SW_L','PA','SW_H'])

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [3]:
G = nx.read_gpickle('email_prediction.txt')

df = pd.DataFrame(index=G.nodes())


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
def salary_predictions():
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    from sklearn.neural_network import MLPClassifier

    df = pd.DataFrame(index=G.nodes())
    df['degree'] = pd.Series(G.degree())
    a,h = nx.hits(G, max_iter=100, tol=1e-08, normalized=True)
    df['Authority'] = pd.Series(a)
    df['Pagerank'] = pd.Series(nx.pagerank(G))
    df['Betw. centrality']=pd.Series(nx.betweenness_centrality(G))
    df['Department']=pd.Series(nx.get_node_attributes(G,'Department'))
    df['ManagementSalary']=pd.Series(nx.get_node_attributes(G,'ManagementSalary'))

    df_train = df[~pd.isnull(df['ManagementSalary'])]
    y = df_train['ManagementSalary']
    X = df_train.iloc[:,:-1]

    df_unknown =  df[pd.isnull(df['ManagementSalary'])]
    df_prob = pd.DataFrame(index=df_unknown.index)
    X_unknown = df_unknown.iloc[:,:-1]

    X = StandardScaler().fit_transform(X)
    X_unknown = StandardScaler().fit_transform(X_unknown)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

    mlp = MLPClassifier(solver = 'lbfgs',hidden_layer_sizes =[20,10,5,5])
    mlp.fit(X,y)
    #mlp.score(X_test,y_test)
    df_prob['Probability']=mlp.predict_proba(X_unknown)[:,1]
    return df_prob.iloc[:,0]

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [4]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [ ]:
def new_connections_predictions():
    
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    from sklearn.neural_network import MLPClassifier
    pref_attach = list(nx.preferential_attachment(G))
    df = pd.DataFrame(index=[(x[0], x[1]) for x in pref_attach])
    df['PA']=[x[2] for x in pref_attach]

    temp = list(nx.resource_allocation_index(G))
    df_temp = pd.DataFrame(index=[(x[0], x[1]) for x in temp])
    df_temp['RAI']=[x[2] for x in temp]
    df = df.merge(df_temp, left_index = True, right_index = True)

    temp = list(nx.jaccard_coefficient(G))
    df_temp = pd.DataFrame(index=[(x[0], x[1]) for x in temp])
    df_temp['JC']=[x[2] for x in temp]
    df = df.merge(df_temp, left_index = True, right_index = True)

    temp = list(nx.cn_soundarajan_hopcroft(G,community = 'Department'))
    df_temp = pd.DataFrame(index=[(x[0], x[1]) for x in temp])
    df_temp['CSH']=[x[2] for x in temp]
    df = df.merge(df_temp, left_index = True, right_index = True)

    temp = list(nx.ra_index_soundarajan_hopcroft(G,community = 'Department'))
    df_temp = pd.DataFrame(index=[(x[0], x[1]) for x in temp])
    df_temp['RISH']=[x[2] for x in temp]
    df = df.merge(df_temp, left_index = True, right_index = True)

    df = df.merge(future_connections, left_index = True, right_index = True)

    df_train = df[~pd.isnull(df['Future Connection'])]
    y = df_train['Future Connection']
    X = df_train.iloc[:,:-1]

    df_unknown =  df[pd.isnull(df['Future Connection'])]
    df_prob = pd.DataFrame(index=df_unknown.index)
    X_unknown = df_unknown.iloc[:,:-1]

    X = StandardScaler().fit_transform(X)
    X_unknown = StandardScaler().fit_transform(X_unknown)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

    mlp = MLPClassifier(solver = 'lbfgs',hidden_layer_sizes =[20,10,5,5])
    mlp.fit(X,y)
    mlp.score(X_test,y_test)

    df_prob = pd.DataFrame(index=df_unknown.index)
    df_prob['Probability']=mlp.predict_proba(X_unknown)[:,1]
    
    return df_prob.iloc[:,0]

(0, 9)          0.030779
(0, 19)         0.058016
(0, 20)         0.240459
(0, 35)         0.019679
(0, 38)         0.013918
(0, 42)         0.817772
(0, 43)         0.016209
(0, 50)         0.012752
(0, 53)         0.549801
(0, 54)         0.040168
(0, 62)         0.362755
(0, 63)         0.051976
(0, 69)         0.028125
(0, 72)         0.012593
(0, 83)         0.164975
(0, 90)         0.026807
(0, 91)         0.017305
(0, 95)         0.111675
(0, 100)        0.031030
(0, 106)        0.972530
(0, 108)        0.018425
(0, 109)        0.012541
(0, 110)        0.012646
(0, 118)        0.012994
(0, 122)        0.027888
(0, 131)        0.038660
(0, 133)        0.743505
(0, 140)        0.065033
(0, 149)        0.142112
(0, 151)        0.013853
                  ...   
(988, 989)      0.012457
(988, 996)      0.012455
(988, 997)      0.032148
(988, 1000)     0.012459
(988, 1002)     0.012455
(989, 994)      0.012456
(989, 996)      0.012456
(989, 1003)     0.012456
(989, 1004)     0.012456
